In [96]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd
import json

In [89]:
# test str
teststr = "The combination of super-fresh ingredients"
tokens = nltk.word_tokenize(teststr.lower())
print(tokens)

['the', 'combination', 'of', 'super-fresh', 'ingredients']


In [90]:
stopword = stopwords.words('english')

# for w in tokens:
#     if w not in stopword:
#         print(w)
        
lst = [k for k in tokens if k not in stopword]
print(lst)

['combination', 'super-fresh', 'ingredients']


In [91]:
df_pred = pd.read_csv('../semEval16/check_correctness/graph_train.csv')
df_pred['Target_matching'] = False
df_pred

,sent_id,text,Source,Target,Polar_expression,Polarity,Target_matching
0,1004293:0,Judging from previous posts this used to be a ...,NaN,NaN,not any longer.,Negative,False
1,1004293:1,"We, there were four of us, arrived at noon - t...",NaN,NaN,-,Positive,False
2,1004293:1,"We, there were four of us, arrived at noon - t...",NaN,NaN,there were,Positive,False
3,1004293:2,"They never brought us complimentary noodles, i...",NaN,NaN,NaN,NaN,False
4,1004293:3,The food was lousy - too sweet or too salty an...,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...
2552,FF#10:7,"I can't believe that it was, but please put th...",NaN,NaN,NaN,NaN,False
2553,FF#10:8,The waitress came to check in on us every few ...,NaN,NaN,NaN,NaN,False
2554,FF#10:9,I couldn't ignore the fact that she reach over...,NaN,NaN,NaN,NaN,False
2555,FF#10:10,She then put the check down without asking if ...,NaN,NaN,NaN,NaN,False


In [92]:
with open('../semEval16/data/train/ABSA16_Restaurants_Train_SB1_v2.json') as f:
    gold = json.load(f)

gold[0:10]

[{'review_id': '1004293',
  'sent_id': '1004293:0',
  'text': 'Judging from previous posts this used to be a good place, but not any longer.',
  'opinions': [{'Source': [[], []],
    'Target': ['place', ['51:56']],
    'Polar_expression': [[], []],
    'Polarity': 'negative',
    'Intensity': '',
    'Category': 'RESTAURANT#GENERAL'}]},
 {'review_id': '1004293',
  'sent_id': '1004293:1',
  'text': 'We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.',
  'opinions': [{'Source': [[], []],
    'Target': ['staff', ['75:80']],
    'Polar_expression': [[], []],
    'Polarity': 'negative',
    'Intensity': '',
    'Category': 'SERVICE#GENERAL'}]},
 {'review_id': '1004293',
  'sent_id': '1004293:2',
  'text': 'They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.',
  'opinions': [{'Source': [[], []],
    'Target': [[], []],
    'Polar_expressi

##### Matching Rule: at least one common word between predicted target and golden target

In [93]:
matching_result = []

for p in range(len(df_pred)):
    id = df_pred.iloc[p]['sent_id']
    matching_result.append(False)
    
    if pd.isnull(df_pred.iloc[p]['Target']) == False:
        
        for g in gold:
            if id == g['sent_id']:

                for o in g['opinions']:
                    if len(o['Target'][0]) > 0:
                        pstr = df_pred.iloc[p]['Target']
                        gstr = o['Target'][0]

                        ptokens = nltk.word_tokenize(pstr.lower())
                        gtokens = nltk.word_tokenize(gstr.lower())

                        plst = [k for k in ptokens if k not in stopword]
                        glst = [m for m in gtokens if m not in stopword]
                        # print(plst, glst)

                        for pelement in plst:
                            if pelement in glst:
                                matching_result[p] = True
                                
            
# for g in gold:
#     for o in g['opinions']:
#         print(o['Target'][0], len(o['Target'][0]))

# for p in range(20):
#     if (pd.isnull(df_pred.iloc[p]['Target'])) == False:
#         print(df_pred.iloc[p]['Target'])

In [94]:
df_pred['Target_matching'] = matching_result
df_pred

,sent_id,text,Source,Target,Polar_expression,Polarity,Target_matching
0,1004293:0,Judging from previous posts this used to be a ...,NaN,NaN,not any longer.,Negative,False
1,1004293:1,"We, there were four of us, arrived at noon - t...",NaN,NaN,-,Positive,False
2,1004293:1,"We, there were four of us, arrived at noon - t...",NaN,NaN,there were,Positive,False
3,1004293:2,"They never brought us complimentary noodles, i...",NaN,NaN,NaN,NaN,False
4,1004293:3,The food was lousy - too sweet or too salty an...,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...
2552,FF#10:7,"I can't believe that it was, but please put th...",NaN,NaN,NaN,NaN,False
2553,FF#10:8,The waitress came to check in on us every few ...,NaN,NaN,NaN,NaN,False
2554,FF#10:9,I couldn't ignore the fact that she reach over...,NaN,NaN,NaN,NaN,False
2555,FF#10:10,She then put the check down without asking if ...,NaN,NaN,NaN,NaN,False


In [95]:
df_pred.to_csv("../semEval16/check_correctness/graph-train-correctness.csv")